In [12]:
%load_ext dotenv
%dotenv


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [13]:
import pandas as pd
import os
import googlemaps
from tqdm import tqdm
tqdm.pandas(desc="Geocoding..")

In [14]:
apikey = os.getenv("GMAP_API_KEY")
assert apikey is not None, "Please set GMAP_API_KEY in .env file"

In [15]:
gmaps = googlemaps.Client(key=apikey)

In [16]:
# create file cache
import joblib

cache=joblib.Memory(".cache", verbose=0)


@cache.cache
def geocode(street_address, subdistrict, district=None, province=None, country="TH"):
    # components = {'administrative_area_level_1': province, 'sublocality_level_1': district, 'sublocality_level_2': subdistrict, 'country': country}
    # # remove None values
    # components = {k: v for k, v in components.items() if v is not None}

    components = {'sublocality_level_2': subdistrict, 'country': country}
    if district is not None:
        components['sublocality_level_1'] = district
    if province is not None:
        components['administrative_area_level_1'] = province

    return gmaps.geocode(street_address, language="th", components=components)




In [17]:
geocode(street_address="โรงเรียนวัดธรรมมงคล (หลวงพ่อวิริยังค์อุปถัมภ์)", subdistrict="พระโขนง", district="บางจาก", province="กรุงเทพมหานคร")

[{'address_components': [{'long_name': '132',
    'short_name': '132',
    'types': ['street_number']},
   {'long_name': 'ซอย ปุณณวิถี',
    'short_name': 'ซอย ปุณณวิถี',
    'types': ['route']},
   {'long_name': 'แขวงบางจาก',
    'short_name': 'แขวงบางจาก',
    'types': ['political', 'sublocality', 'sublocality_level_2']},
   {'long_name': 'เขตพระโขนง',
    'short_name': 'เขตพระโขนง',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': 'กรุงเทพมหานคร',
    'short_name': 'กรุงเทพมหานคร',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'ประเทศไทย',
    'short_name': 'TH',
    'types': ['country', 'political']},
   {'long_name': '10260', 'short_name': '10260', 'types': ['postal_code']}],
  'formatted_address': '132 ซอย ปุณณวิถี แขวงบางจาก เขตพระโขนง กรุงเทพมหานคร 10260 ประเทศไทย',
  'geometry': {'location': {'lat': 13.6885237, 'lng': 100.6154805},
   'location_type': 'ROOFTOP',
   'viewport': {'northeast': {'lat': 13.6897808302

In [18]:
df = pd.read_csv('ballot_complete_clean.csv.gz')

In [19]:
df.columns

Index(['รหัสจังหวัด', 'จังหวัด', 'รหัสสำนักทะเบียน', 'สำนักทะเบียน',
       'รหัสตำบล', 'ตำบล', 'เขตเลือกตั้ง', 'หน่วยเลือกตั้ง',
       'สถานที่เลือกตั้ง', 'อธิบายเพิ่มเติมสถานที่เลือกตั้ง', 'NER'],
      dtype='object')

In [22]:
sample = df.sample(n=100)

In [23]:
sample['geocoded_obj'] = sample.progress_apply(lambda x: geocode(street_address=x['สถานที่เลือกตั้ง'], subdistrict=x["ตำบล"], province=x["จังหวัด"]), axis=1)

Geocoding..: 100%|██████████| 100/100 [00:10<00:00,  9.24it/s]


In [24]:
# check it is empty list
sample.geocoded_obj.apply(lambda x: len(x)).value_counts()

,รหัสจังหวัด,จังหวัด,รหัสสำนักทะเบียน,สำนักทะเบียน,รหัสตำบล,ตำบล,เขตเลือกตั้ง,หน่วยเลือกตั้ง,สถานที่เลือกตั้ง,อธิบายเพิ่มเติมสถานที่เลือกตั้ง,NER,geocoded_obj
5389,10,กรุงเทพมหานคร,1043,ท้องถิ่นเขตคันนายาว,104301,คันนายาว,15,33,เต็นท์บริเวณลานกีฬาหมู่บ้านแสงอรุณ,NaN,เต็นท์บริเวณ<FACILITY>ลานกีฬาหมู่บ้านแสงอรุณ</...,[{'address_components': [{'long_name': 'หมู่บ้...
23981,30,จังหวัดนครราชสีมา,3024,อำเภอโนนแดง,302404,วังหิน,7,1,ศาลากลางบ้านโนนเพ็ด,NaN,ศาลากลาง<LOCATION>บ้านโนนเพ็ด</LOCATION>,[{'address_components': [{'long_name': 'บ้านโน...
19787,26,จังหวัดนครนายก,2602,อำเภอปากพลี,260202,เกาะโพธิ์,1,3,วัดสะแกซึง,NaN,<FACILITY>วัดสะแกซึง</FACILITY>,"[{'address_components': [{'long_name': '1', 's..."
46810,42,จังหวัดเลย,4285,ท้องถิ่นเทศบาลตำบลเขาแก้ว,420304,เขาแก้ว,4,9,ศาลาประชาคม,NaN,ศาลาประชาคม,[{'address_components': [{'long_name': 'VCR2+C...
34734,34,จังหวัดอุบลราชธานี,3420,อำเภอตาลสุม,342004,หนองกุง,3,4,ศาลาวัดบ้านคำแคน หมู่ที่ 3,NaN,<FACILITY>ศาลาวัดบ้านคําแคน</FACILITY> หมู่ที่ 3,[{'address_components': [{'long_name': 'ประเทศ...
...,...,...,...,...,...,...,...,...,...,...,...,...
31181,33,จังหวัดศรีสะเกษ,3307,อำเภอปรางค์กู่,330702,กู่,7,9,ศาลาประชาคมบ้านพอกน้อย,NaN,ศาลาประชาคมบ้านพอกน้อย,[{'address_components': [{'long_name': 'VX82+V...
73187,65,จังหวัดพิษณุโลก,6507,อำเภอวัดโบสถ์,650703,ท้อแท้,5,2,ศาลาอเนกประสงค์ SML,NaN,ศาลาอเนกประสงค์ <unk>,[{'address_components': [{'long_name': 'ประเทศ...
42395,40,จังหวัดขอนแก่น,4015,อำเภอหนองสองห้อง,401510,ดอนดั่ง,10,8,ศาลาประชาคมบ้านหนองแวงน้อย,NaN,<FACILITY>ศาลาประชาคมบ้านหนองแวงน้อย</FACILITY>,[{'address_components': [{'long_name': 'อำเภอ ...
84427,80,จังหวัดนครศรีธรรมราช,8013,อำเภอร่อนพิบูลย์,801301,ร่อนพิบูลย์,5,11,ศาลาประจำหมู่บ้าน หมู่ที่ 8 ตำบลร่อนพิบูลย์,NaN,ศาลาประจําหมู่บ้าน หมู่ที่ 8 <LOCATION>ตําบลร่...,[{'address_components': [{'long_name': 'หมู่ที...
